***
## 📋 Index <a class="anchor"><a id='index'></a>
    
> ## [<font color='MediumPurple'> __1. Pre Information__</font>](#one)
>> ### [<font color='White'> __1.1 Libraries__</font>](#oneone)
>> ### [<font color='White'> __1.2 Metatrader Info__</font>](#onetwo)
>> ### [<font color='White'> __1.3 YFinance Info__</font>](#onethree)
> ## [<font color='MediumPurple'> __2. VARIABLES__ </font>](#two)
> ## [<font color='MediumPurple'> __3. BACKTESTINGS__ </font>](#three)
>> ### [<font color='White'> __3.1 EMA Crossover__</font>](#threeone)
>> ### [<font color='White'> __3.2 Bollinger Bands__</font>](#threetwo)
>> ### [<font color='White'> __3.3 RSI__</font>](#threethree)
>> ### [<font color='white'> __3.5 Fixed Range Volume Profile (FRVP)__</font>](#threefive)
>> ### [<font color='white'> __3.6 Base Signal__</font>](#threesix)
>>> #### [<font color='white'> __3.6.1 Candles Base Signal__</font>](#threesixone)
>>> #### [<font color='white'> __3.6.2 Candles Engulfing__</font>](#threesixtwo)
>>> #### [<font color='white'> __3.6.3 Candles Morning Star__</font>](#threesixthree)
>>> #### [<font color='white'> __3.6.4 Candles Evening Star__</font>](#threesixfour)
>>> #### [<font color='white'> __3.6.5 Candles Hammer__</font>](#threesixfive)
>>> #### [<font color='white'> __3.6.6 Candles Hanging Man__</font>](#threesixsix)


>> ### [<font color='white'> __3.7 Aplicação de Algoritmos Unsupervised__</font>](#threesix)



___

>> ### [<font color='white'> __3.8 Fibonacci Levels__</font>](#threeeight)

> ## [<font color='MediumPurple'> __4. BACKTESTINGS CROSSOVERS__ </font>](#four)
>> ### [<font color='white'> __4.1 EMA + Bollinger Bands__</font>](#fourone)
>> ### [<font color='white'> __4.2 EMA + Bollinger Bands + RSI__</font>](#fourtwo)



## <font color='white'> __1. Pre Information__ </font> <a class="anchor" id="one"></a>

[<font color='mediumpurple'>Back to Index </font>](#index)


#### <font color='white'> __1.1 Libraries__ </font> <a class="anchor" id="oneone"></a>

[<font color='mediumpurple'>Back to Index </font>](#index)


In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import pandas_ta as pta
import MetaTrader5 as mt5
import plotly.express as px
from scipy import stats, signal
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from scipy.signal import find_peaks
from backtesting import Strategy, Backtest
from datetime import time, datetime, timezone

# Turn off the warning
pd.options.mode.chained_assignment = None  # or 'warn' to display the warning

#### <font color='white'> __1.2 Metatrader Info__ </font> <a class="anchor" id="onetwo"></a>

[<font color='mediumpurple'>Back to Index </font>](#index)

In [2]:
# Start the plataform with initialize()
mt5.initialize()

# User credentials

login = 51116412
password = '39sz2vL3'
server = 'ICMarketsSC-Demo'

# log in in the server account of MT5
mt5.login(login, password, server)

True


#### <font color='white'> __1.3 YFinance Info__ </font> <a class="anchor" id="onethree"></a>

[<font color='mediumpurple'>Back to Index </font>](#index)

#### __Analisar commodities são mais sasonais__

In [3]:
## Full

# Get a list of all available symbols
symbols = mt5.symbols_get()

# Create a DataFrame without using append
tickers = pd.DataFrame(
    [[symbol.name, symbol.description, f"{symbol.currency_base}/{symbol.currency_profit}", f"{symbol.path}"] for symbol in symbols],
    columns=['Symbol', 'Description', 'Currency', 'Path']
)

# tickers = tickers.head(20).copy()
# tickers.style

In [4]:
tickers['Type'] = None

for i in tickers['Path']:
    if "Majors" in i:
        tickers.loc[tickers['Path'] == i, 'Type'] = 'Major'
    elif "Minors" in i:
        tickers.loc[tickers['Path'] == i, 'Type'] = 'Minor'
    elif "Exotic" in i:
        tickers.loc[tickers['Path'] == i, 'Type'] = 'Exotic'
    elif "Commodities" in i:
        tickers.loc[tickers['Path'] == i, 'Type'] = 'Commodity'
    else:
        tickers.loc[tickers['Path'] == i, 'Type'] = None


In [5]:
# Set the ticker
ticker = 'EURUSD'

In [6]:
# ## If Stock
# data = yf.Ticker(ticker).history(
#                                     period="2y",
#                                     interval="1d"
#                                     )

# data.reset_index(inplace=True)
# data.rename(columns={'Date': 'time', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Adj Close': 'adj_close', 'Volume': 'volume'}, inplace=True)



In [7]:
# If Cambial
data = pd.DataFrame(mt5.copy_rates_from_pos(ticker, mt5.TIMEFRAME_D1, 0, 10000))[['time', 'open', 'high', 'low', 'close','tick_volume']]
data.rename(columns={'tick_volume': 'volume'}, inplace=True)

def convert_timestamp_to_datetime(timestamp):
    dt_object = datetime.fromtimestamp(timestamp, tz=timezone.utc)
    return dt_object.strftime("%Y-%m-%d %H:%M:%S %Z")

# Apply the function to the 'time' column
data['time'] = data['time'].apply(convert_timestamp_to_datetime)
data['time'] = pd.to_datetime(data['time'])
#  data.set_index('time', inplace=True)

* Os datasets apresentados contêm 1000 velas de uma hora, 1000 horas o que resulta no historico dos ultimos 41 dias ou 1 mês e 1 semana.
Não se utilizou mais historico com o intuito de tornar a analise mais sensivel a alterações de comportamentos do mercado (de referir que analise fundamental tem um peso teorico de 1/3 de decisão de abertura de posição, posteriormente pode ser implementado um processo de NLP para detectar a noticias e outras correlações fundamentais para suportar a decisão) [EM ESTUDO]  

## <font color='white'> __2. VARIABLES__ </font> <a class="anchor" id="two"></a>

[<font color='mediumpurple'>Back to Index </font>](#index)

In [8]:
SYMBOL = ticker #'EURUSD'        
MAGIC = 100 # Ref
VOLUME = 0.02 # Size of lot
START_POSITION = 0
NUMBER_BARS = 1000

# VOLATILITY FUNCTIONS
    # SMA Bands parameters      
SMA_PERIOD = 14 # SMA 14
STANDARD_DEVIATION_SMA = 2 # Deviation of 2
    # RSI parameters
RSI_PERIOD = 14
OVERBOUGHT_LEVEL = 70
OVERSOLD_LEVEL = 30
MIDDLE_LEVEL = 50
        # Harmonic Patterns
err_allowed = 10.0/100

## <font color='white'> __3. BACKTESTING__ </font> <a class="anchor" id="three"></a>

[<font color='mediumpurple'>Back to Index </font>](#index)

### <font color='white'> __3.1 EMA Crossover__ </font> <a class="anchor" id="threeone"></a>

[<font color='mediumpurple'>Back to Index </font>](#index)


* This is used has trend indicator

In [9]:
def get_emas_crossover(data: pd.DataFrame, l1: int, l2: int, l3: int):
    '''
    This function measures the crossover of 3 EMAs, 13, 48 and 200.
    
    Fundamental:
        * As long as the price remains above the chosen EMA level, the trader remains on the buy side, 
        if the price falls below the level of the selected EMA, the trader is a seller unless the price 
        crosses to the upside of the EMA.
        * When SMAs crossover between themself the result is in the conditions
        * We use the "Golden Cross" approach studied by The Dow Jones Industrial where concludes that the crossover between 13days 
        and 48.5 days EMAs is the most profitable
        * 48% Success Rate
    
    Conditions:
        * Buy = 13 > 48 > 200
        * Sell = 13 < 48 < 200
        * Flat = Else
    '''
    
    data['ema1'] = pta.ema(data.close, length=l1)
    data['ema2'] = pta.ema(data.close, length=l2)
    data['ema3'] = pta.ema(data.close, length=l3)
    
    for i, row in data.iterrows():
        if row['ema1'] > row['ema2'] > row['ema3']:
            data.at[i, 'crossover_signal'] = 1
            data.at[i, 'crossover_signal_positions'] = row['close'] - 0.01
        elif row['ema1'] < row['ema2'] < row['ema3']:
            data.at[i, 'crossover_signal'] = -1
            data.at[i, 'crossover_signal_positions'] = row['close'] + 0.01
        else:
            data.at[i, 'crossover_signal'] = 0
            data.at[i, 'crossover_signal_positions'] = None

    # fig = go.Figure(data=[go.Candlestick(x=data.time,
    #             open=data.open,
    #             high=data.high,
    #             low=data.low,
    #             close=data.close,
    #             name = 'Candlestick chart'),
    #             go.Scatter(x=data.time, y=data.EMA1, line=dict(color='black', width=1), name=f'Short-Term (n = {l1})'), 
    #             go.Scatter(x=data.time, y=data.EMA2, line=dict(color='blue', width=1), name=f'Medium-Term (n = {l2})'),
    #             go.Scatter(x=data.time, y=data.EMA3, line=dict(color='orange', width=1), name=f'Long-Term (n = {l3})'),
    #             ])


    # # fig.add_scatter(x=data.index, y=data['Crossover_Signal_Positions'], mode="markers",
    # #                 marker=dict(size=5, color=data['Crossover_Signal'].apply(lambda x: 'green' if x == 1 else 'red' if x == -1 else 'rgba(0,0,0,0)')),
    # #                 name="Final Signal",
    # #                 text=data['Crossover_Signal'].apply(lambda x: 'Buy' if x == 1 else 'Sell' if x == -1 else '')
    # # )

    # fig.update_yaxes(fixedrange=False)
    
    # fig.update_layout(title=dict(text=f'EMA Crossover on {ticker} in Daily Timeframe', x=.5),
    #                     xaxis_rangeslider_visible=False
    #                     )

    # fig.show()
        
    return data

# Assuming you have a DataFrame named 'data'
get_emas_crossover(data, 14, 48, 200)


,time,open,high,low,close,volume,ema1,ema2,ema3,crossover_signal,crossover_signal_positions
0,1985-07-26 00:00:00+00:00,0.68170,0.68300,0.67970,0.68190,641,NaN,NaN,NaN,0.0,None
1,1985-07-29 00:00:00+00:00,0.69180,0.69270,0.68980,0.69150,551,NaN,NaN,NaN,0.0,None
2,1985-07-30 00:00:00+00:00,0.69310,0.69460,0.69020,0.69300,871,NaN,NaN,NaN,0.0,None
3,1985-07-31 00:00:00+00:00,0.69950,0.70080,0.69850,0.69940,451,NaN,NaN,NaN,0.0,None
4,1985-08-01 00:00:00+00:00,0.69590,0.70150,0.69280,0.69570,1721,NaN,NaN,NaN,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...
9995,2024-02-13 00:00:00+00:00,1.07711,1.07949,1.07008,1.07091,56846,1.079523,1.084593,1.080482,0.0,None
9996,2024-02-14 00:00:00+00:00,1.07075,1.07346,1.06951,1.07276,53500,1.078621,1.084110,1.080405,0.0,None
9997,2024-02-15 00:00:00+00:00,1.07249,1.07849,1.07240,1.07722,51996,1.078434,1.083828,1.080374,0.0,None
9998,2024-02-16 00:00:00+00:00,1.07650,1.07876,1.07321,1.07763,50485,1.078327,1.083575,1.080346,0.0,None


### <font color='white'> __3.2 Bollinger Bands__ </font> <a class="anchor" id="threetwo"></a>

[<font color='mediumpurple'>Back to Index </font>](#index)



In [10]:
def get_sma_bands(data: pd.DataFrame, l1: int, std: int):
    '''
    This function measures the SMA Bollinger Bands interval

    Fundamental:
        * Bollinger Bands are envelopes plotted at a standard deviation level above and below a simple 
        moving average of the price. Because the distance of the bands is based on standard deviation,
        they adjust to volatility swings in the underlying price.
        * Identify sharp, short-term price movements and potential entry and exit points.
        * Bollinger Bands are able to capture about 90% of the price action in a given asset or cryptocurrency.
        * 95% Success Rate
        * Period = 14
        * Standard Deviation = 2

    Conditions:
        * Buy = Last Close Price <= Lower Band
        * Sell = Last Close Price >= Upper Band
        * Flat = Else  
    '''
    bars_sma_bb_df = pta.bbands(data.close, length=l1, std=std)
    data = data.join(bars_sma_bb_df)

    data.rename(columns={f'BBU_{l1}_{std}.0': 'bb_upperband'}, inplace=True)
    data.rename(columns={f'BBL_{l1}_{std}.0': 'bb_lowerband'}, inplace=True)
    data.rename(columns={f'BBM_{l1}_{std}.0': 'bb_middleband'}, inplace=True)
    data.rename(columns={f'BBB_{l1}_{std}.0': 'bb_bandwidth'}, inplace=True)
    data.rename(columns={f'BBP_{l1}_{std}.0': 'bb_percent'}, inplace=True)


    for i, row in data.iterrows():
        if row['bb_lowerband'] > row['close']:
            data.at[i, 'bbands_signal'] = 1
            data.at[i, 'bbands_signal_positions'] = row['close'] - 0.01
        elif row['bb_upperband'] < row['close']:
            data.at[i, 'bbands_signal'] = -1
            data.at[i, 'bbands_signal_positions'] = row['close'] + 0.01
        else:
            data.at[i, 'bbands_signal'] = 0 
            data.at[i, 'bbands_signal_positions'] = None
    
    # fig = go.Figure(data=[go.Candlestick(x=data.time,
    #             open=data.open,
    #             high=data.high,
    #             low=data.low,
    #             close=data.close,
    #             name = 'Candlestick chart'),
    #             go.Scatter(x=data.time, y=data['BBM_14_2.0'], line=dict(color='red', width=1), name=f"Middle Band (n={l1}, std={std})"),
    #             go.Scatter(x=data.time, y=data['BBL_14_2.0'], line=dict(color='blue', width=1), name=f"Lower Band (n={l1}, std={std})"),
    #             go.Scatter(x=data.time, y=data['BBU_14_2.0'], line=dict(color='blue', width=1), name=f"Upper Band (n={l1}, std={std})")])

    # fig.add_scatter(x=data.time, y=data['BBands_Signal_Positions'], mode="markers",
    #                 marker=dict(size=5, 
    #                             color= data.apply(lambda row: 'green' if row['BBands_Signal'] == 1 else ('red' if row['BBands_Signal'] == -1 else 'grey'), axis=1)),
    #                 name="Signal",
    #                 text= data.apply(lambda row: 'Sell' if row['BBands_Signal'] == -1 else ('Buy' if row['BBands_Signal'] == 1 else ''), axis=1))

    # fig.update_layout(title=dict(text=f'Bollinger Bands on {ticker} in Daily Timeframe', x=.5),
    #             xaxis_rangeslider_visible=False,
    #             )

    # fig.show()

    return data

get_sma_bands(data, 14, 2)

,time,open,high,low,close,volume,ema1,ema2,ema3,crossover_signal,crossover_signal_positions,bb_lowerband,bb_middleband,bb_upperband,bb_bandwidth,bb_percent,bbands_signal,bbands_signal_positions
0,1985-07-26 00:00:00+00:00,0.68170,0.68300,0.67970,0.68190,641,NaN,NaN,NaN,0.0,None,NaN,NaN,NaN,NaN,NaN,0.0,None
1,1985-07-29 00:00:00+00:00,0.69180,0.69270,0.68980,0.69150,551,NaN,NaN,NaN,0.0,None,NaN,NaN,NaN,NaN,NaN,0.0,None
2,1985-07-30 00:00:00+00:00,0.69310,0.69460,0.69020,0.69300,871,NaN,NaN,NaN,0.0,None,NaN,NaN,NaN,NaN,NaN,0.0,None
3,1985-07-31 00:00:00+00:00,0.69950,0.70080,0.69850,0.69940,451,NaN,NaN,NaN,0.0,None,NaN,NaN,NaN,NaN,NaN,0.0,None
4,1985-08-01 00:00:00+00:00,0.69590,0.70150,0.69280,0.69570,1721,NaN,NaN,NaN,0.0,None,NaN,NaN,NaN,NaN,NaN,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2024-02-13 00:00:00+00:00,1.07711,1.07949,1.07008,1.07091,56846,1.079523,1.084593,1.080482,0.0,None,1.070562,1.079794,1.089027,1.710038,0.018855,0.0,None
9996,2024-02-14 00:00:00+00:00,1.07075,1.07346,1.06951,1.07276,53500,1.078621,1.084110,1.080405,0.0,None,1.069465,1.078949,1.088433,1.758027,0.173702,0.0,None
9997,2024-02-15 00:00:00+00:00,1.07249,1.07849,1.07240,1.07722,51996,1.078434,1.083828,1.080374,0.0,None,1.069573,1.078359,1.087144,1.629409,0.435201,0.0,None
9998,2024-02-16 00:00:00+00:00,1.07650,1.07876,1.07321,1.07763,50485,1.078327,1.083575,1.080346,0.0,None,1.069609,1.077951,1.086292,1.547655,0.480776,0.0,None


### <font color='white'> __3.3 RSI__ </font> <a class="anchor" id="threethree"></a>

[<font color='mediumpurple'>Back to Index </font>](#index)

In [11]:
def get_rsi(data : pd.DataFrame, l1 : int, oblvl : int, oslvl : int):
    '''
    This function measures the Relative Strenght Index canal
    
    Fundamental:
        * The relative strength index (RSI) is a momentum indicator used in technical analysis. 
        RSI measures the speed and magnitude of a security's recent price changes to evaluate
        overvalued or undervalued conditions in the price of that security.
        * 90% Success Rate
        * Period = 14
        * Overbought = 70%
        * Oversold = 30%
        * Range = Last 1000 Candles 

    Conditions:
        * Buy = RSI value <= OVERSOLD_LEVEL
        * Sell = RSI value >= OVERBOUGHT_LEVEL
        * Flat = Else   
    '''
    data['rsi'] = pta.rsi(data.close, length=l1)

    for i, row in data.iterrows():
        if row['rsi'] <= oslvl:
            data.at[i, 'rsi_signal'] = 1
            data.at[i, 'rsi_signal_positions'] = row['close'] - 0.01
        elif row['rsi'] >= oblvl:
            data.at[i, 'rsi_signal'] = -1
            data.at[i, 'rsi_signal_positions'] = row['close'] + 0.01
        else: 
            data.at[i, 'rsi_signal'] = 0
            data.at[i, 'rsi_signal_positions'] = None

    # fig = go.Figure(data=[go.Candlestick(x=data.time,
    #             open=data['open'],
    #             high=data['high'],
    #             low=data['low'],
    #             close=data['close'],
    #             name = 'Candlestick chart'),])

    # fig.add_scatter(x=data.time,
    #                 y=data['RSI_Signal_Positions'], 
    #                 mode="markers",
    #                 marker=dict(size=5, 
    #                             color= data.apply(lambda row: 'green' if row['RSI_Signal'] == 1 else 'red', axis=1)),
    #                 name="Signal",
    #                 text= data.apply(lambda row: 'Sell' if row['RSI_Signal'] == -1 else ('Buy' if row['RSI_Signal'] == 1 else ''), axis=1))

    # fig.update_layout(title=dict(text=f'RSI on {ticker} in Daily Timeframe', x=.5),
    #                 xaxis_rangeslider_visible=False
    #                 )

    # fig1 = go.Figure(go.Scatter(x=data.time, y=data['RSI'], line=dict(color='blue', width=1), name="RSI"))

    # fig1.add_trace(go.Scatter(x=data.time, y=[30] * len(data.index), line=dict(color='red', width=1), name="OVERSOLD_LEVEL"))
    # fig1.add_trace(go.Scatter(x=data.time, y=[50] * len(data.index), line=dict(color='red', width=1), name="MIDDLE_LEVEL"))
    # fig1.add_trace(go.Scatter(x=data.time, y=[70] * len(data.index), line=dict(color='red', width=1), name="OVERBOUGHT_LEVEL"))

    # # Add a title to the plot
    # fig1.update_layout(title=dict(text=f'RSI on {ticker} in Daily Timeframe', x=.5),
    #                 xaxis_rangeslider_visible=False)

    # fig.show()
    # fig1.show()

    return data

get_rsi(data, 14, 70, 30)

,time,open,high,low,close,volume,ema1,ema2,ema3,crossover_signal,crossover_signal_positions,rsi,rsi_signal,rsi_signal_positions
0,1985-07-26 00:00:00+00:00,0.68170,0.68300,0.67970,0.68190,641,NaN,NaN,NaN,0.0,None,NaN,0.0,None
1,1985-07-29 00:00:00+00:00,0.69180,0.69270,0.68980,0.69150,551,NaN,NaN,NaN,0.0,None,NaN,0.0,None
2,1985-07-30 00:00:00+00:00,0.69310,0.69460,0.69020,0.69300,871,NaN,NaN,NaN,0.0,None,NaN,0.0,None
3,1985-07-31 00:00:00+00:00,0.69950,0.70080,0.69850,0.69940,451,NaN,NaN,NaN,0.0,None,NaN,0.0,None
4,1985-08-01 00:00:00+00:00,0.69590,0.70150,0.69280,0.69570,1721,NaN,NaN,NaN,0.0,None,NaN,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2024-02-13 00:00:00+00:00,1.07711,1.07949,1.07008,1.07091,56846,1.079523,1.084593,1.080482,0.0,None,33.860576,0.0,None
9996,2024-02-14 00:00:00+00:00,1.07075,1.07346,1.06951,1.07276,53500,1.078621,1.084110,1.080405,0.0,None,36.933346,0.0,None
9997,2024-02-15 00:00:00+00:00,1.07249,1.07849,1.07240,1.07722,51996,1.078434,1.083828,1.080374,0.0,None,43.721616,0.0,None
9998,2024-02-16 00:00:00+00:00,1.07650,1.07876,1.07321,1.07763,50485,1.078327,1.083575,1.080346,0.0,None,44.314994,0.0,None


### <font color='white'> __3.4 Market Session__ </font> <a class="anchor" id="threefour"></a>

[<font color='mediumpurple'>Back to Index </font>](#index)

In [12]:
# # Define your market sessions
# market_sessions = {
#     'Sydney': (time(21, 0), time(23, 59)),
#     'Sydney_Tokyo': (time(0, 0), time(5, 59)),
#     'Tokyo': (time(6, 0), time(6, 59)),
#     'Tokyo_London': (time(7, 0), time(8, 59)),
#     'London': (time(9, 0), time(11, 59)),
#     'London-NYork': (time(12, 0), time(15, 59)),
#     'NYork': (time(16, 0), time(20, 59))
# }

# # Convert the 'time' column to datetime objects
# data['time'] = pd.to_datetime(data['time'])

# # Initialize the 'Market_Session' column with None
# data['Market_Session'] = None

# # Loop through the DataFrame and set 'Market_Session' based on time
# for i, row in data.iterrows():
#     timestamp = row['time'].time()  # Extract time from datetime
#     for session, (start_time, end_time) in market_sessions.items():
#         if start_time <= timestamp <= end_time:
#             data.at[i, 'Market_Session'] = session
#             break  # Break the inner loop if a session is found

# data.tail(3)

### <font color='white'> __3.5 Fixed Range Volume Profile (FRVP)__</font> <a class="anchor" id="threefive"></a>

[<font color='mediumpurple'>Back to Index </font>](#index)

https://medium.com/swlh/how-to-analyze-volume-profiles-with-python-3166bb10ff24

* Once this metric is made to run on daily and weekly basis in order to combine both trend, p.e. 4th day of the week, 4 frvp's combining a trend crossover with the weekly trend in order to reinforce the confirmation or not.   


In [13]:
# dataa = data.tail(40)
# fig = go.Figure()
# fig.add_trace(go.Histogram(name='Vol Profile',
#                         x=dataa.close,  ## Ver porque é que não se usa resultados em vez dos dados
#                         y=dataa.volume, 
#                         nbinsx=len(data), 
#                         histfunc='sum', 
#                         histnorm='probability density', #'density', #'percent', #'probability', #'probability density',
#                         marker_color='#B0C4DE'))

# fig.update_layout(
#         title=dict(text='Fixed Range Volume Profile (Daily over 2 Months)', x=0.5),
#         xaxis_title="Average of Close",
#         yaxis_title="Probability Density of Volume",
#         xaxis=dict(zeroline=False, showgrid=False), 
#         yaxis=dict(zeroline=False, showgrid=False)
#         )

In [14]:
# def get_volume_profile(data):

#         '''
#         Fundamentals:

#         * This function measure the probability density of a event fall into the corresponding bin getting peaks and vallyes of the gaussian funtion. 
#         Firstly the last 2 months are analized on a daily candle prespective and secondly the last 2 months on a weekly prespective.

#         * Is used as measure of frequency of price reached which is result of *probability density*, 
#         the area of each bar corresponds to the probability that an event will fall into the corresponding bin (here, the sum of all bin AREAS equals 1).
        
#         * Kernel density estimation is a way to estimate the probability density function (PDF) of a random variable in a non-parametric way. 
#         gaussian_kde works for both uni-variate and multi-variate data.
        
#         * The strategy is to act in High Volume Nodes, "where the volume is high is where the price moves out" 
#         Conditions:

#         __kde_factor__ : [0;1], the highest he value the smooth the line of p.density of volume and less peaks and valleys are detected. 
#         Is used a default of 0.02 for daily analizes and 0.04 for weekly analizes, this values are choosed by testing several values until get the most relaible.   
#         https://aakinshin.net/posts/kde-bw/
#         '''

#         current_close_value = data.tail(1).close.iloc[0]
        
#         # Daily 
#         kde_factor = .04 #.02
#         num_samples = 500
#         kde = stats.gaussian_kde(data.close, weights=data.volume, bw_method=kde_factor)
#         xr = np.linspace(data.close.min(), data.close.max(),num_samples)
#         kdy = kde(xr)

#         # Find Peaks of the distribution
#         peaks, _ = find_peaks(kdy, prominence=1)

#         # Alternativa de Maximos e Minimos 
#                 # max_idx = list(argrelextrema(price.values, comparator=np.greater, order=1)[0])
#                 # min_idx = list(argrelextrema(price.values, comparator=np.less, order=1)[0])

#         close_avg_peak = xr[peaks]
#         volume_pd_peak = kdy[peaks]
        
#         peak_df = pd.DataFrame({'close_avg': close_avg_peak, 'volume_prob_d': volume_pd_peak})

#         peak_df['dif_rate'] = current_close_value - peak_df['close_avg']
#         support = peak_df[peak_df['dif_rate'] > 0].close_avg.max() 
#         resistence = peak_df[peak_df['dif_rate'] < 0].close_avg.min() 
#         ## Estes pontos são as primeiras zonas de resistencia e support utilizados como stoploss baseados em comportamentos do volumes (retalho)

#         peak_df = peak_df.sort_values(by='close_avg', ascending=True)
#         peak_df.reset_index(inplace=True, drop=True)

#         peak_df['peakvall_order'] = peak_df['close_avg'].rank(method='first')
#         peak_df['poc_order'] = peak_df['volume_prob_d'].rank(method='first', ascending=False)


#         # VIZUALIZATION
#         # FRVP daily chart
#         fig = go.Figure()
#         fig.add_trace(go.Histogram(name='Vol Profile',
#                                 x=data.close,  ## Ver porque é que não se usa resultados em vez dos dados
#                                 y=data.volume, 
#                                 nbinsx=num_samples, 
#                                 histfunc='sum', 
#                                 histnorm='probability density', #'density', #'percent', #'probability', #'probability density',
#                                 marker_color='#B0C4DE'))
                        
#         fig.add_trace(go.Scatter(name='KDE',
#                                 x=xr, 
#                                 y=kdy, 
#                                 mode='lines', 
#                                 marker_color='#D2691E'))

#         fig.add_trace(go.Scatter(name="Peaks", 
#                                 x=close_avg_peak, 
#                                 y=volume_pd_peak, 
#                                 mode='markers', 
#                                 marker=dict(size=10, color="MediumPurple")))

#         fig.add_trace(go.Scatter(name="Poc", 
#                                 x=peak_df[peak_df['poc_order'] == 1].close_avg, 
#                                 y=peak_df[peak_df['poc_order'] == 1].volume_prob_d, 
#                                 mode='markers',
#                                 text=[peak_df[peak_df['poc_order'] == 1].poc_order],
#                                 marker=dict(color='LightSkyBlue',
#                                                 size=10,
#                                                 line=dict(color='blue',
#                                                         width=1)))
#         )

#         fig.update_layout(
#                 title=dict(text='Fixed Range Volume Profile (Daily over 2 Months)', x=0.5),
#                 xaxis_title="Average of Close",
#                 yaxis_title="Probability Density of Volume",
#                 xaxis=dict(zeroline=False, showgrid=False), 
#                 yaxis=dict(zeroline=False, showgrid=False)
#                 )

#         fig.add_vrect(name="Current Close Price",
#                         x0=current_close_value,
#                         x1=current_close_value,
#                         fillcolor='orange',
#                         opacity=1,
#                         line_width=1
#                         )
        
#         # fig.add_vrect(name="Support Line",
#         #                 x0=support,
#         #                 x1=support,
#         #                 fillcolor='orange',
#         #                 opacity=1,
#         #                 line_width=3
#         #                 )
        
#         # fig.add_vrect(name="Resistence Line",
#         #                 x0=resistence,
#         #                 x1=resistence,
#         #                 fillcolor='orange',
#         #                 opacity=1,
#         #                 line_width=3
#         #                 )
        
#         fig.show()

#         print('Daily Support : {}\nDaily Resistence : {}\nCurrent Price : {}'.format(support, resistence, current_close_value))
#         return current_close_value, resistence, support, peak_df

# current_close_value, resistence, support, peak_df = get_volume_profile(data)

In [15]:
# data_2024 = data[data['time'].dt.year == 2024] 
# data_2023 = data[data['time'].dt.year == 2023] 

In [16]:
# plt.scatter(x=data_2023.time,
#             y=data_2023.close,
#             label='Data Points',
#             color='blue',
#             marker='x',
#             linestyle='-')

# # Add labels and a title
# plt.xlabel('Time')
# plt.ylabel('Close')
# plt.title('Simple Scatter Plot')


In [17]:
# plt.scatter(x=data_2023.volume,
#             y=data_2023.close,
#             label='Data Points',
#             color='blue',
#             marker='x',
#             linestyle='-')

# # Add labels and a title
# plt.xlabel('Volume')
# plt.ylabel('Close')
# plt.title('Simple Scatter Plot')

In [18]:
# from sklearn.cluster import KMeans

# X = data_2023[['close']].values

# # Define the number of clusters (you can adjust this)
# n_clusters = 6

# # Apply K-Means clustering
# kmeans = KMeans(n_clusters=n_clusters, random_state=42)
# data_2023['cluster_kmeans'] = kmeans.fit_predict(X)

# plt.figure(figsize=(15, 5))
# # Plot the clustered data Scatted
# plt.subplot(1, 2, 1)
# plt.scatter(data_2023['volume'], data_2023['close'], c=data_2023['cluster_kmeans'], cmap='viridis', marker='x')
# plt.xlabel('Time')
# plt.ylabel('Close')
# plt.title('KMeans Clustered Scatter Plot')

# # Plot the clustered data Lined
# plt.subplot(1, 2, 2)
# plt.scatter(data_2023['time'], data_2023['close'], c=data_2023['cluster_kmeans'], cmap='viridis', marker='x')
# plt.xlabel('Time')
# plt.ylabel('Close')
# plt.title('KMeans Clustered Line Plot')

# # Add colorbar for cluster labels
# cbar = plt.colorbar()
# cbar.set_label('KMeans Cluster Label')

# plt.show()

In [19]:
# from sklearn.cluster import DBSCAN

# dbscan = DBSCAN(eps=0.001,
#                 min_samples=5)

# data_2023['cluster_dbscan'] = dbscan.fit_predict(X)

# plt.figure(figsize=(15, 5))
# # Plot the clustered data Scatted
# plt.subplot(1, 2, 1)
# plt.scatter(data_2023['volume'], data_2023['close'], c=data_2023['cluster_dbscan'], cmap='viridis', marker='x')
# plt.xlabel('Time')
# plt.ylabel('Close')
# plt.title('KMeans Clustered Scatter Plot')

# # Plot the clustered data Lined
# plt.subplot(1, 2, 2)
# plt.scatter(data_2023['time'], data_2023['close'], c=data_2023['cluster_dbscan'], cmap='viridis', marker='x')
# plt.xlabel('Time')
# plt.ylabel('Close')
# plt.title('DBSCAN Clustered Line Plot')

# # Add colorbar for cluster labels
# cbar = plt.colorbar()
# cbar.set_label('DBSCAN Cluster Label')

# plt.show()

### <font color='white'> __3.6 Candles Patterns__</font> <a class="anchor" id="threesix"></a>

[<font color='mediumpurple'>Back to Index </font>](#index)


#### <font color='white'> __3.6.1 Base Signal__</font> <a class="anchor" id="threesixone"></a>

[<font color='mediumpurple'>Back to Index </font>](#index)

In [20]:
def get_candle_base_pattern(data : pd.DataFrame):
    '''
    This function perform a for loop measure the candles types, bearish ou bullish and shadows size comparatively with body size.
    
    Conditions:
        * bullish = Close Price > Open Price
        * bearish = Close Price < Open Price
        * flat = Else

    '''
    for i, row in data.iterrows():
        if row['close'] > row['open']:
            data.at[i, 'candle_signal'] = 1
        elif row['close'] < row['open']:
            data.at[i, 'candle_signal'] = -1
        else:
            data.at[i, 'candle_signal'] = 0

    # # For vizualization of basic pattern
    # start_date = '2024-01-14'
    # end_date = '2024-01-27'

    # dataa = data[(data['time'] >= start_date) & (data['time'] <= end_date)]

    # fig = go.Figure(data=[go.Candlestick(x=dataa.time,
    #             open=dataa['open'],
    #             high=dataa['high'],
    #             low=dataa['low'],
    #             close=dataa['close'],
    #             name = 'Candlestick Chart')
    #             ])

    # fig.update_layout(title=dict(text=f'Candlestick Chart of {ticker} in Daily Timeframe', x=.5),
    #                 xaxis_rangeslider_visible=False)

    # fig.show()
    
    return data

get_candle_base_pattern(data)

,time,open,high,low,close,volume,ema1,ema2,ema3,crossover_signal,crossover_signal_positions,rsi,rsi_signal,rsi_signal_positions,candle_signal
0,1985-07-26 00:00:00+00:00,0.68170,0.68300,0.67970,0.68190,641,NaN,NaN,NaN,0.0,None,NaN,0.0,None,1.0
1,1985-07-29 00:00:00+00:00,0.69180,0.69270,0.68980,0.69150,551,NaN,NaN,NaN,0.0,None,NaN,0.0,None,-1.0
2,1985-07-30 00:00:00+00:00,0.69310,0.69460,0.69020,0.69300,871,NaN,NaN,NaN,0.0,None,NaN,0.0,None,-1.0
3,1985-07-31 00:00:00+00:00,0.69950,0.70080,0.69850,0.69940,451,NaN,NaN,NaN,0.0,None,NaN,0.0,None,-1.0
4,1985-08-01 00:00:00+00:00,0.69590,0.70150,0.69280,0.69570,1721,NaN,NaN,NaN,0.0,None,NaN,0.0,None,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2024-02-13 00:00:00+00:00,1.07711,1.07949,1.07008,1.07091,56846,1.079523,1.084593,1.080482,0.0,None,33.860576,0.0,None,-1.0
9996,2024-02-14 00:00:00+00:00,1.07075,1.07346,1.06951,1.07276,53500,1.078621,1.084110,1.080405,0.0,None,36.933346,0.0,None,1.0
9997,2024-02-15 00:00:00+00:00,1.07249,1.07849,1.07240,1.07722,51996,1.078434,1.083828,1.080374,0.0,None,43.721616,0.0,None,1.0
9998,2024-02-16 00:00:00+00:00,1.07650,1.07876,1.07321,1.07763,50485,1.078327,1.083575,1.080346,0.0,None,44.314994,0.0,None,1.0


#### <font color='white'> __3.6.2 Engulfing Pattern__</font> <a class="anchor" id="threesixtwo"></a>

[<font color='mediumpurple'>Back to Index </font>](#index)


In [21]:
def get_engulfing(data : pd.DataFrame):
    '''
    This function detect if the last candle is a bearish or bulish engolfing
    
    Fundamental:
        * In a Bullish Engulfing pattern, the body of the second (bullish) candle should completely engulf the 
        body of the first (bearish) candle. This means that the opening price of the second candle is lower than
        the closing price of the first candle, and the closing price of the second candle is higher than the opening
        price of the first candle.
        * In a Bearish Engulfing pattern, the body of the second (bearish) candle should completely engulf the 
        body of the first (bullish) candle. This means that the opening price of the second candle is higher than
        the closing price of the first candle, and the closing price of the second candle is lower than the opening
        price of the first candle.
        * Bulish engulfing has a reversal rate of 63%. That means price closes above the top of the candlestick pattern 63% of the time.
        * Bearish performing that way 79% of the time.
    '''

    # Iterate through DataFrame rows
    for i in range(2, len(data)):
        last_candle = data.loc[i]
        second_last_candle = data.loc[i - 1]
        third_last_candle = data.loc[i - 2]

        # View
        if (
            second_last_candle['candle_signal'] == 1 and third_last_candle['candle_signal'] == -1
            and second_last_candle['open'] < third_last_candle['close']
            and second_last_candle['close'] > third_last_candle['open']
            and (last_candle['low'] > second_last_candle['low']) & (last_candle['low'] > third_last_candle['low'])
        ):
            data.at[i, 'engulfing_candle'] = 1
            data.at[i, 'engulfing_candle_position'] = last_candle['open'] + 0.01

        elif (
            second_last_candle['candle_signal'] == -1 and third_last_candle['candle_signal'] == 1
            and second_last_candle['open'] > third_last_candle['close']
            and second_last_candle['close'] < third_last_candle['open']
            and (last_candle['high'] < second_last_candle['high']) & (last_candle['high'] < third_last_candle['high'])
        ):
            data.at[i, 'engulfing_candle'] = -1
            data.at[i, 'engulfing_candle_position'] = last_candle['open'] - 0.01

        else:
            data.at[i, 'engulfing_candle'] = 0
            data.at[i, 'engulfing_candle_position'] = None


    # fig = go.Figure(data=[go.Candlestick(x=data.time,
    #             open=data['open'],
    #             high=data['high'],
    #             low=data['low'],
    #             close=data['close'],
    #             name = 'Candlestick Chart')
    #             ])

    # fig.add_scatter(x=data.time,
    #             y=data['engulfing_candle_position'], 
    #             mode="markers",
    #             marker=dict(size=5, color= data.apply(lambda row: 'green' if row['engulfing_candle'] == 1 else 'red', axis=1)),
    #             name="Engulfing Signal",
    #             text= data.apply(lambda row: 'Sell' if row['engulfing_candle'] == -1 else ('Buy' if row['engulfing_candle'] == 1 else ''), axis=1))

    # fig.update_layout(title=dict(text=f'Engulfing Candles on {ticker} in Daily Timeframe', x=.5),
    #                 xaxis_rangeslider_visible=False
    #                 )

    # fig.show()

    return data

get_engulfing(data)

,time,open,high,low,close,volume,ema1,ema2,ema3,crossover_signal,crossover_signal_positions,rsi,rsi_signal,rsi_signal_positions,candle_signal,engulfing_candle,engulfing_candle_position
0,1985-07-26 00:00:00+00:00,0.68170,0.68300,0.67970,0.68190,641,NaN,NaN,NaN,0.0,None,NaN,0.0,None,1.0,NaN,NaN
1,1985-07-29 00:00:00+00:00,0.69180,0.69270,0.68980,0.69150,551,NaN,NaN,NaN,0.0,None,NaN,0.0,None,-1.0,NaN,NaN
2,1985-07-30 00:00:00+00:00,0.69310,0.69460,0.69020,0.69300,871,NaN,NaN,NaN,0.0,None,NaN,0.0,None,-1.0,0.0,NaN
3,1985-07-31 00:00:00+00:00,0.69950,0.70080,0.69850,0.69940,451,NaN,NaN,NaN,0.0,None,NaN,0.0,None,-1.0,0.0,NaN
4,1985-08-01 00:00:00+00:00,0.69590,0.70150,0.69280,0.69570,1721,NaN,NaN,NaN,0.0,None,NaN,0.0,None,-1.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2024-02-13 00:00:00+00:00,1.07711,1.07949,1.07008,1.07091,56846,1.079523,1.084593,1.080482,0.0,None,33.860576,0.0,None,-1.0,0.0,None
9996,2024-02-14 00:00:00+00:00,1.07075,1.07346,1.06951,1.07276,53500,1.078621,1.084110,1.080405,0.0,None,36.933346,0.0,None,1.0,0.0,None
9997,2024-02-15 00:00:00+00:00,1.07249,1.07849,1.07240,1.07722,51996,1.078434,1.083828,1.080374,0.0,None,43.721616,0.0,None,1.0,0.0,None
9998,2024-02-16 00:00:00+00:00,1.07650,1.07876,1.07321,1.07763,50485,1.078327,1.083575,1.080346,0.0,None,44.314994,0.0,None,1.0,0.0,None


#### <font color='white'> __3.6.3 Candles Morning Star__</font> <a class="anchor" id="threesixthree"></a>

[<font color='mediumpurple'>Back to Index </font>](#index)


In [22]:
def morning_star(data : pd.DataFrame):
    '''
    A morning star is a three-candle pattern with the low point on the second candle. 
    However, the low point is only apparent after the close of the third candle.
    High volume on the third day is often seen as a confirmation of the pattern 
    (and a subsequent uptrend) regardless of other indicators.
    
    Fundamental:
        * First candle: The first candlestick is a tall bearish candle, in line with the ongoing downward price swing. It shows that the downswing is still intact.
        * Second candle: The second candlestick is a small-bodied candle, typically a doji or a spinning top. It gaps down from the first candlestick. It can be of 
        any color, but the most important thing is that it is small and gaps below the first, in anticipation of the continuation of the existing downswing. 
        Its typical small size suggests indecision in the market. If it is a doji, the pattern is called Morning Doji Star.
        * Third candle: The third candle is bullish and quite sizeable. It opens below the second candle and closes above the midpoint of the first candle. 
        * Bullish reversal 78% of the time.
        * Bearish reversal of the upward price trend, and testing reveals that it does 71% of the time.
    '''

    # Iterate through DataFrame rows
    for i in range(2, len(data)):
        last_candle = data.loc[i]
        second_last_candle = data.loc[i - 1]
        third_last_candle = data.loc[i - 2]

        # Daily View
        if(
            third_last_candle['candle_signal'] == -1
            and (second_last_candle['open'] < third_last_candle['close']) & (second_last_candle['close'] < third_last_candle['close'])   
            and last_candle['candle_signal'] == 1
            and second_last_candle['open'] < last_candle['open']) & (second_last_candle['close'] < last_candle['open']
        ):
            data.at[i, 'morning_star_candle'] = 1
            data.at[i, 'morning_star_candle_position'] = last_candle['close'] + 0.01
        
        else:
            data.at[i, 'morning_star_candle'] = 0
            data.at[i, 'morning_star_candle_position'] = None
        
    # fig = go.Figure(data=[go.Candlestick(x=data.time,
    #             open=data['open'],
    #             high=data['high'],
    #             low=data['low'],
    #             close=data['close'],
    #             name = 'Candlestick Chart')
    #             ])

    # fig.add_scatter(x=data.time,
    #             y=data['morning_star_candle_position'], 
    #             mode="markers",
    #             marker=dict(size=5, color= data.apply(lambda row: 'green' if row['morning_star_candle'] == 1 else 'red', axis=1)),
    #             name="Morning Star Signal",
    #             text='Buy')

    # fig.update_layout(title=dict(text=f'Morning Star Candles on {ticker} in Daily Timeframe', x=.5),
    #                 xaxis_rangeslider_visible=False
    #                 )
    
    # fig.show()

    return data

morning_star(data)

,time,open,high,low,close,volume,ema1,ema2,ema3,crossover_signal,crossover_signal_positions,rsi,rsi_signal,rsi_signal_positions,candle_signal,engulfing_candle,engulfing_candle_position,morning_star_candle,morning_star_candle_position
0,1985-07-26 00:00:00+00:00,0.68170,0.68300,0.67970,0.68190,641,NaN,NaN,NaN,0.0,None,NaN,0.0,None,1.0,NaN,NaN,NaN,NaN
1,1985-07-29 00:00:00+00:00,0.69180,0.69270,0.68980,0.69150,551,NaN,NaN,NaN,0.0,None,NaN,0.0,None,-1.0,NaN,NaN,NaN,NaN
2,1985-07-30 00:00:00+00:00,0.69310,0.69460,0.69020,0.69300,871,NaN,NaN,NaN,0.0,None,NaN,0.0,None,-1.0,0.0,NaN,0.0,NaN
3,1985-07-31 00:00:00+00:00,0.69950,0.70080,0.69850,0.69940,451,NaN,NaN,NaN,0.0,None,NaN,0.0,None,-1.0,0.0,NaN,0.0,NaN
4,1985-08-01 00:00:00+00:00,0.69590,0.70150,0.69280,0.69570,1721,NaN,NaN,NaN,0.0,None,NaN,0.0,None,-1.0,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2024-02-13 00:00:00+00:00,1.07711,1.07949,1.07008,1.07091,56846,1.079523,1.084593,1.080482,0.0,None,33.860576,0.0,None,-1.0,0.0,None,0.0,None
9996,2024-02-14 00:00:00+00:00,1.07075,1.07346,1.06951,1.07276,53500,1.078621,1.084110,1.080405,0.0,None,36.933346,0.0,None,1.0,0.0,None,0.0,None
9997,2024-02-15 00:00:00+00:00,1.07249,1.07849,1.07240,1.07722,51996,1.078434,1.083828,1.080374,0.0,None,43.721616,0.0,None,1.0,0.0,None,0.0,None
9998,2024-02-16 00:00:00+00:00,1.07650,1.07876,1.07321,1.07763,50485,1.078327,1.083575,1.080346,0.0,None,44.314994,0.0,None,1.0,0.0,None,0.0,None


#### <font color='white'> __3.6.4 Candles Evening Star__</font> <a class="anchor" id="threesixfour"></a>

[<font color='mediumpurple'>Back to Index </font>](#index)

In [23]:
def evening_star(data : pd.DataFrame):
    '''
    WIP
    '''

    # Iterate through DataFrame rows
    for i in range(2, len(data)):
        last_candle = data.loc[i]
        second_last_candle = data.loc[i - 1]
        third_last_candle = data.loc[i - 2]

        # Daily View
        if(
            third_last_candle['candle_signal'] == 1
            and (second_last_candle['open'] > third_last_candle['close']) & (second_last_candle['close'] > third_last_candle['close'])   
            and last_candle['candle_signal'] == -1
            and second_last_candle['open'] > last_candle['open']) & (second_last_candle['close'] > last_candle['open']
        ):
            data.at[i, 'evening_star'] = -1
            data.at[i, 'evening_star_position'] = last_candle['open'] + 0.01
        
        else:
            data.at[i, 'evening_star'] = 0
            data.at[i, 'evening_star_position'] = None
    
    # fig = go.Figure(data=[go.Candlestick(x=data.time,
    #             open=data['open'],
    #             high=data['high'],
    #             low=data['low'],
    #             close=data['close'],
    #             name = 'Candlestick Chart')
    #             ])

    # fig.add_scatter(x=data.time,
    #             y=data['evening_star_position'], 
    #             mode="markers",
    #             marker=dict(size=5, color='red'),
    #             name="Evening Star Signal",
    #             text='Sell')

    # fig.update_layout(title=dict(text=f'Evening Star Candles on {ticker} in Daily Timeframe', x=.5),
    #                 xaxis_rangeslider_visible=False
    #                 )
    
    # fig.show()

    return data

evening_star(data)

,time,open,high,low,close,volume,ema1,ema2,ema3,crossover_signal,...,rsi,rsi_signal,rsi_signal_positions,candle_signal,engulfing_candle,engulfing_candle_position,morning_star_candle,morning_star_candle_position,evening_star,evening_star_position
0,1985-07-26 00:00:00+00:00,0.68170,0.68300,0.67970,0.68190,641,NaN,NaN,NaN,0.0,...,NaN,0.0,None,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1985-07-29 00:00:00+00:00,0.69180,0.69270,0.68980,0.69150,551,NaN,NaN,NaN,0.0,...,NaN,0.0,None,-1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1985-07-30 00:00:00+00:00,0.69310,0.69460,0.69020,0.69300,871,NaN,NaN,NaN,0.0,...,NaN,0.0,None,-1.0,0.0,NaN,0.0,NaN,0.0,NaN
3,1985-07-31 00:00:00+00:00,0.69950,0.70080,0.69850,0.69940,451,NaN,NaN,NaN,0.0,...,NaN,0.0,None,-1.0,0.0,NaN,0.0,NaN,0.0,NaN
4,1985-08-01 00:00:00+00:00,0.69590,0.70150,0.69280,0.69570,1721,NaN,NaN,NaN,0.0,...,NaN,0.0,None,-1.0,0.0,NaN,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2024-02-13 00:00:00+00:00,1.07711,1.07949,1.07008,1.07091,56846,1.079523,1.084593,1.080482,0.0,...,33.860576,0.0,None,-1.0,0.0,None,0.0,None,0.0,None
9996,2024-02-14 00:00:00+00:00,1.07075,1.07346,1.06951,1.07276,53500,1.078621,1.084110,1.080405,0.0,...,36.933346,0.0,None,1.0,0.0,None,0.0,None,0.0,None
9997,2024-02-15 00:00:00+00:00,1.07249,1.07849,1.07240,1.07722,51996,1.078434,1.083828,1.080374,0.0,...,43.721616,0.0,None,1.0,0.0,None,0.0,None,0.0,None
9998,2024-02-16 00:00:00+00:00,1.07650,1.07876,1.07321,1.07763,50485,1.078327,1.083575,1.080346,0.0,...,44.314994,0.0,None,1.0,0.0,None,0.0,None,0.0,None


#### <font color='white'> __3.6.5 Candles Hammer__</font> <a class="anchor" id="threesixfive"></a>

[<font color='mediumpurple'>Back to Index </font>](#index)

In [24]:
def hammer(data : pd.DataFrame):
    '''
    '''

    # Iterate through DataFrame rows
    for i in range(4, len(data)):
        last_candle = data.loc[i]
        second_last_candle = data.loc[i - 1]
        third_last_candle = data.loc[i - 2]
        fourth_last_candle = data.loc[i - 3]
        fiveth_last_candle = data.loc[i - 4]

        if(
            last_candle['candle_signal'] == 1 
            and ((last_candle['open'] - last_candle['low'])/(last_candle['close'] - last_candle['open']) > 2)
            and last_candle['low'] < second_last_candle['low']
            and last_candle['low'] < third_last_candle['low']
            and last_candle['low'] < fourth_last_candle['low']
            and last_candle['low'] < fiveth_last_candle['low']
            and (((last_candle['close'] - last_candle['open'])/(last_candle['high'] - last_candle['close'])) > 3)  
        ):
            data.at[i, 'hammer_candle'] = 1
            data.at[i, 'hammer_candle_position'] = last_candle['low'] - 0.005
        
        else:
            data.at[i, 'hammer_candle'] = 0
            data.at[i, 'hammer_candle_position'] = None
    
    # fig = go.Figure(data=[go.Candlestick(x=data.time,
    #             open=data['open'],
    #             high=data['high'],
    #             low=data['low'],
    #             close=data['close'],
    #             name = 'Candlestick Chart')
    #             ])

    # fig.add_scatter(x=data.time,
    #             y=data['hammer_candle_position'], 
    #             mode="markers",
    #             marker=dict(size=5, color='green'),
    #             name="Hammer Signal",
    #             text='Buy')

    # fig.update_layout(title=dict(text=f'Hammer Candles on {ticker} in Daily Timeframe', x=.5),
    #                 xaxis_rangeslider_visible=False
    #                 )
    
    # fig.show()

    return data

hammer(data)        

C:\Users\joaom\AppData\Local\Temp\ipykernel_21536\1309497825.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  and (((last_candle['close'] - last_candle['open'])/(last_candle['high'] - last_candle['close'])) > 3)


,time,open,high,low,close,volume,ema1,ema2,ema3,crossover_signal,...,rsi_signal_positions,candle_signal,engulfing_candle,engulfing_candle_position,morning_star_candle,morning_star_candle_position,evening_star,evening_star_position,hammer_candle,hammer_candle_position
0,1985-07-26 00:00:00+00:00,0.68170,0.68300,0.67970,0.68190,641,NaN,NaN,NaN,0.0,...,None,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1985-07-29 00:00:00+00:00,0.69180,0.69270,0.68980,0.69150,551,NaN,NaN,NaN,0.0,...,None,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1985-07-30 00:00:00+00:00,0.69310,0.69460,0.69020,0.69300,871,NaN,NaN,NaN,0.0,...,None,-1.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN
3,1985-07-31 00:00:00+00:00,0.69950,0.70080,0.69850,0.69940,451,NaN,NaN,NaN,0.0,...,None,-1.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN
4,1985-08-01 00:00:00+00:00,0.69590,0.70150,0.69280,0.69570,1721,NaN,NaN,NaN,0.0,...,None,-1.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2024-02-13 00:00:00+00:00,1.07711,1.07949,1.07008,1.07091,56846,1.079523,1.084593,1.080482,0.0,...,None,-1.0,0.0,None,0.0,None,0.0,None,0.0,None
9996,2024-02-14 00:00:00+00:00,1.07075,1.07346,1.06951,1.07276,53500,1.078621,1.084110,1.080405,0.0,...,None,1.0,0.0,None,0.0,None,0.0,None,0.0,None
9997,2024-02-15 00:00:00+00:00,1.07249,1.07849,1.07240,1.07722,51996,1.078434,1.083828,1.080374,0.0,...,None,1.0,0.0,None,0.0,None,0.0,None,0.0,None
9998,2024-02-16 00:00:00+00:00,1.07650,1.07876,1.07321,1.07763,50485,1.078327,1.083575,1.080346,0.0,...,None,1.0,0.0,None,0.0,None,0.0,None,0.0,None


#### <font color='white'> __3.6.6 Candles Hanging Man__</font> <a class="anchor" id="threesixsix"></a>

[<font color='mediumpurple'>Back to Index </font>](#index)


In [25]:
def hanging_man(data : pd.DataFrame):
    '''
    '''

    # Iterate through DataFrame rows
    for i in range(5, len(data)):
        last_candle = data.loc[i]
        second_last_candle = data.loc[i - 1]
        third_last_candle = data.loc[i - 2]
        fourth_last_candle = data.loc[i - 3]
        fiveth_last_candle = data.loc[i - 4]
        sixth_last_candle = data.loc[i - 5]

        if(
            second_last_candle['candle_signal'] == 1 
            and ((second_last_candle['open'] - second_last_candle['low'])/(second_last_candle['close'] - second_last_candle['open']) > 2)
            and second_last_candle['high'] > third_last_candle['high']
            and second_last_candle['high'] > fourth_last_candle['high']
            and second_last_candle['high'] > fiveth_last_candle['high']
            and second_last_candle['high'] > sixth_last_candle['high']
            and (((second_last_candle['close'] - second_last_candle['open'])/(second_last_candle['high'] - second_last_candle['close'])) > 3)  
            and last_candle['candle_signal'] == -1 
        ):
            data.at[i, 'hanging_candle'] = -1
            data.at[i, 'hanging_candle_position'] = second_last_candle['high'] + 0.005
        
        else:
            data.at[i, 'hanging_candle'] = 0
            data.at[i, 'hanging_candle_position'] = None 
    
    # fig = go.Figure(data=[go.Candlestick(x=data.time,
    #             open=data['open'],
    #             high=data['high'],
    #             low=data['low'],
    #             close=data['close'],
    #             name = 'Candlestick Chart')
    #             ])

    # fig.add_scatter(x=data.time,
    #             y=data['hanging_candle_position'], 
    #             mode="markers",
    #             marker=dict(size=5, color='red'),
    #             name="Hanging Signal",
    #             text='Sell')

    # fig.update_layout(title=dict(text=f'Hanging Candles on {ticker} in Daily Timeframe', x=.5),
    #                 xaxis_rangeslider_visible=False
    #                 )
    
    # fig.show()

    return data

hanging_man(data)        

C:\Users\joaom\AppData\Local\Temp\ipykernel_21536\1899365233.py:21: RuntimeWarning: divide by zero encountered in double_scalars
  and (((second_last_candle['close'] - second_last_candle['open'])/(second_last_candle['high'] - second_last_candle['close'])) > 3)


,time,open,high,low,close,volume,ema1,ema2,ema3,crossover_signal,...,engulfing_candle,engulfing_candle_position,morning_star_candle,morning_star_candle_position,evening_star,evening_star_position,hammer_candle,hammer_candle_position,hanging_candle,hanging_candle_position
0,1985-07-26 00:00:00+00:00,0.68170,0.68300,0.67970,0.68190,641,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1985-07-29 00:00:00+00:00,0.69180,0.69270,0.68980,0.69150,551,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1985-07-30 00:00:00+00:00,0.69310,0.69460,0.69020,0.69300,871,NaN,NaN,NaN,0.0,...,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,1985-07-31 00:00:00+00:00,0.69950,0.70080,0.69850,0.69940,451,NaN,NaN,NaN,0.0,...,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,1985-08-01 00:00:00+00:00,0.69590,0.70150,0.69280,0.69570,1721,NaN,NaN,NaN,0.0,...,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2024-02-13 00:00:00+00:00,1.07711,1.07949,1.07008,1.07091,56846,1.079523,1.084593,1.080482,0.0,...,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None
9996,2024-02-14 00:00:00+00:00,1.07075,1.07346,1.06951,1.07276,53500,1.078621,1.084110,1.080405,0.0,...,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None
9997,2024-02-15 00:00:00+00:00,1.07249,1.07849,1.07240,1.07722,51996,1.078434,1.083828,1.080374,0.0,...,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None
9998,2024-02-16 00:00:00+00:00,1.07650,1.07876,1.07321,1.07763,50485,1.078327,1.083575,1.080346,0.0,...,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None


### <font color='white'> __3.7 Supervised Algorithms Test__</font> <a class="anchor" id="threeseven"></a>

[<font color='mediumpurple'>Back to Index </font>](#index)

* Linear Regression
* Logistic Regression
* Decision Trees
* Random Forest
* Support Vector Machines (SVM)
* K-Nearest Neighbors (KNN)
* Naive Bayes
* Gradient Boosting Machines (GBM)
* Neural Networks (Multilayer Perceptron)
* AdaBoost
* Extreme Gradient Boosting (XGBoost)
* LightGBM
* CatBoost
* Lasso Regression
* Ridge Regression

In [26]:
import xgboost as xgb
from sklearn.svm import SVC, SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier, BaggingClassifier

from sklearn import metrics

In [27]:
def supervised_alg_predict(data : pd.DataFrame, label_column):
    '''
    This function test in a range of the given list of supervised models the performance of each.
    '''

    # Drop NaNs
    df = data.drop(columns=data.filter(regex='position').columns).copy()
    df = df.drop(columns='time').copy()
    df = df.dropna()

    # Split label from data
    X = df.drop(label_column, axis=1)
    y = df[label_column]

    # Define the seed
    seed = 0

    # Spliting the data, leaving 20% in the hold out sample
    X_train, X_test, y_train, y_test = train_test_split(
                                                        X,
                                                        y,
                                                        random_state = seed,
                                                        shuffle = True,
                                                        stratify = y,
                                                        test_size = .2,
                                                    )

    # Checkpoint print of data split
    print('Train sample size: \033[1m{}%\033[0m | Test sample size: \033[1m{}%\033[0m'.format(round(len(y_train)/len(y)*100), round(len(y_test)/len(y)*100)))    


    ### MODELING ###
    
    Logistic_model = LogisticRegression(random_state=seed).fit(X_train, y_train) # Logistic Regression
    DecisionTree_model = DecisionTreeClassifier(random_state=seed).fit(X_train, y_train) # Decision Tree
    RandomForest_model = RandomForestClassifier(random_state=seed).fit(X_train, y_train) # Random Forest
    SupportVectorC_model = SVC(random_state=seed).fit(X_train, y_train) # Support Vector Classification
    KNN_model = KNeighborsClassifier().fit(X_train, y_train) # K-Nearest Neighbors
    Gaussian_model = GaussianNB().fit(X_train, y_train) # Gaussian Naive Bayes
    GradientBoosting_model = GradientBoostingClassifier(random_state=seed).fit(X_train, y_train) # Gradient Boosting
    MLP_model = MLPClassifier(random_state=seed).fit(X_train, y_train) # Neural Network
    AdaBoost_model =  AdaBoostClassifier(random_state = seed).fit(X_train, y_train) # Adaptative Boosting
    
        
    # Create a list of your models
    models = [
                Logistic_model, 
                DecisionTree_model, 
                RandomForest_model,
                SupportVectorC_model, 
                KNN_model, 
                Gaussian_model,
                GradientBoosting_model, 
                MLP_model, 
                AdaBoost_model, 
            ]

    # Creating DataFrame for results
    results_df = pd.DataFrame(
        columns=['Model', 'F1-Score', 'Accuracy', 'Precision', 'Recall'])

    for model in models:

        y_pred = model.predict(X_test)

        # Benchmark metrics
        f1 = metrics.f1_score(y_test, y_pred, average='micro')
        accuracy = metrics.accuracy_score(y_test, y_pred)
        precision = metrics.precision_score(y_test, y_pred, average='micro')
        recall = metrics.recall_score(y_test, y_pred, average='micro')

        # Append results to DataFrame
        results_df = pd.concat([results_df, pd.DataFrame({
                                                            'Model': str(model),  # Convert model to string for better readability
                                                            'F1-Score': f1,
                                                            'Accuracy': accuracy,
                                                            'Precision': precision,
                                                            'Recall': recall
                                                        }, index=[model])])

    results_df.set_index('Model', inplace=True)

    return results_df, X_train, X_test, y_train, y_test

In [28]:
results_df, X_train, X_test, y_train, y_test = supervised_alg_predict(data, 'candle_signal')

Train sample size: 80% | Test sample size: 20%


C:\Users\joaom\AppData\Local\Temp\ipykernel_21536\2775777576.py:73: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame({


In [29]:
results_df

,F1-Score,Accuracy,Precision,Recall
Model,,,,
LogisticRegression(random_state=0),0.504844,0.504844,0.504844,0.504844
DecisionTreeClassifier(random_state=0),0.655788,0.655788,0.655788,0.655788
RandomForestClassifier(random_state=0),0.733299,0.733299,0.733299,0.733299
SVC(random_state=0),0.495156,0.495156,0.495156,0.495156
KNeighborsClassifier(),0.501785,0.501785,0.501785,0.501785
GaussianNB(),0.481387,0.481387,0.481387,0.481387
GradientBoostingClassifier(random_state=0),0.712902,0.712902,0.712902,0.712902
MLPClassifier(random_state=0),0.497705,0.497705,0.497705,0.497705
AdaBoostClassifier(random_state=0),0.582866,0.582866,0.582866,0.582866


In [30]:

# Drop NaNs
df = data.drop(columns=data.filter(regex='position').columns).copy()
df = df.drop(columns='time').copy()
df = df.dropna()

# Split label from data
X = df.drop('candle_signal', axis=1)
y = df['candle_signal']

# Define the seed
seed = 0

# Spliting the data, leaving 20% in the hold out sample
X_train, X_test, y_train, y_test = train_test_split(
                                                    X,
                                                    y,
                                                    random_state = seed,
                                                    shuffle = True,
                                                    stratify = y,
                                                    test_size = .2,
                                                )

GradientBoosting_model = GradientBoostingClassifier(random_state=seed).fit(X_train, y_train)
RandomForest_model = RandomForestClassifier(random_state=seed).fit(X_train, y_train)
DecisionTree_model = DecisionTreeClassifier(random_state=seed).fit(X_train, y_train)

predictionsGrad = GradientBoosting_model.predict(X_test)
predictionsRandomForest = RandomForest_model.predict(X_test)
predictionsDecisionTree = DecisionTree_model.predict(X_test)
predictionsGrad

array([ 1.,  1., -1., ..., -1.,  1., -1.])

In [31]:
X_test['predictionsGrad'] = predictionsGrad
X_test['predictionsRandomForest'] = predictionsRandomForest
X_test['predictionsDecisionTree'] = predictionsDecisionTree

X_test

,open,high,low,close,volume,ema1,ema2,ema3,crossover_signal,rsi,rsi_signal,engulfing_candle,morning_star_candle,evening_star,hammer_candle,hanging_candle,predictionsGrad,predictionsRandomForest,predictionsDecisionTree
3040,1.14630,1.16230,1.14500,1.15910,2181,1.139881,1.149748,1.214391,-1.0,61.500359,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.0
7368,1.36854,1.37018,1.36797,1.36889,15775,1.368597,1.361129,1.336983,1.0,54.113425,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
5057,1.30350,1.30390,1.29390,1.29760,9250,1.305542,1.311872,1.268256,0.0,38.870806,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0
3596,1.03260,1.03630,1.02900,1.03620,8427,1.037824,1.050893,1.094852,-1.0,44.879415,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
1036,1.00800,1.01160,1.00330,1.00880,1581,1.022501,1.022531,1.041160,-1.0,41.349421,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,1.04070,1.04110,1.04030,1.04080,151,1.042403,1.051646,1.076107,-1.0,43.582911,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
6252,1.45630,1.46120,1.45020,1.45790,10279,1.437116,1.421602,1.388222,1.0,67.167839,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
1367,1.32110,1.32640,1.31490,1.31510,1701,1.318318,1.296866,1.223230,1.0,53.386450,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0
6922,1.30961,1.31330,1.30330,1.31088,48962,1.319295,1.319740,1.343220,-1.0,43.012103,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0


In [32]:
X_test_result = pd.merge(X_test, data[['candle_signal']], left_index=True, right_index=True)
X_test_result

,open,high,low,close,volume,ema1,ema2,ema3,crossover_signal,rsi,rsi_signal,engulfing_candle,morning_star_candle,evening_star,hammer_candle,hanging_candle,predictionsGrad,predictionsRandomForest,predictionsDecisionTree,candle_signal
3040,1.14630,1.16230,1.14500,1.15910,2181,1.139881,1.149748,1.214391,-1.0,61.500359,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.0,1.0
7368,1.36854,1.37018,1.36797,1.36889,15775,1.368597,1.361129,1.336983,1.0,54.113425,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
5057,1.30350,1.30390,1.29390,1.29760,9250,1.305542,1.311872,1.268256,0.0,38.870806,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0
3596,1.03260,1.03630,1.02900,1.03620,8427,1.037824,1.050893,1.094852,-1.0,44.879415,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
1036,1.00800,1.01160,1.00330,1.00880,1581,1.022501,1.022531,1.041160,-1.0,41.349421,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,1.04070,1.04110,1.04030,1.04080,151,1.042403,1.051646,1.076107,-1.0,43.582911,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6252,1.45630,1.46120,1.45020,1.45790,10279,1.437116,1.421602,1.388222,1.0,67.167839,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
1367,1.32110,1.32640,1.31490,1.31510,1701,1.318318,1.296866,1.223230,1.0,53.386450,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0
6922,1.30961,1.31330,1.30330,1.31088,48962,1.319295,1.319740,1.343220,-1.0,43.012103,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0


In [33]:
def get_volume_profile(data):

        '''
        Fundamentals:

        * This function measure the probability density of a event fall into the corresponding bin getting peaks and vallyes of the gaussian funtion. 
        * Is used as measure of frequency of price reached which is result of *probability density*, 
        the area of each bar corresponds to the probability that an event will fall into the corresponding bin (here, the sum of all bin AREAS equals 1).
        Once histogram can lead to a perfect and accurate detection of peaks and vallyes due bins is applied kernel density estimation.
        * Kernel density estimation is a way to estimate the probability density function (PDF) of a random variable in a non-parametric way. 
        gaussian_kde works for both uni-variate and multi-variate data.
        * The strategy is to act in High Volume Nodes, "where the volume is high is where the price moves out" 
        
        Conditions:

        __kde_factor__ : [0;1], the highest he value the smooth the line of p.density of volume and less peaks and valleys are detected. 
        Is used a default of 0.02 for daily analizes and 0.04 for weekly analizes, this values are choosed by testing several values until get the most relaible.   
        https://aakinshin.net/posts/kde-bw/
        '''

        current_close_value = data.tail(1).close.iloc[0]
        
        # Daily 
        kde_factor = .01 #.02
        step = 500
        kde = stats.gaussian_kde(data.close, weights=data.volume, bw_method=kde_factor)
        xr = np.linspace(data.close.min(), data.close.max(),step)
        kdy = kde(xr)

        # Find Peaks of the distribution
        peaks, _ = find_peaks(kdy, prominence=1)

        close_avg_peak = xr[peaks]
        volume_pd_peak = kdy[peaks]
        
        peak_df = pd.DataFrame({'close_avg': close_avg_peak, 'volume_prob_d': volume_pd_peak})


        peak_df = peak_df.sort_values(by='close_avg', ascending=True)
        peak_df.reset_index(inplace=True, drop=True)

        peak_df['peakvall_order'] = peak_df['close_avg'].rank(method='first')
        peak_df['poc_order'] = peak_df['volume_prob_d'].rank(method='first', ascending=False)


        # VIZUALIZATION
        # FRVP daily chart
        fig = go.Figure()
        fig.add_trace(go.Histogram(name='Vol Profile',
                                x=data.close,  ## Ver porque é que não se usa resultados em vez dos dados
                                y=data.volume, 
                                nbinsx=step, 
                                histfunc='sum', 
                                histnorm='probability density', #'density', #'percent', #'probability', #'probability density',
                                marker_color='#B0C4DE'))
                        
        fig.add_trace(go.Scatter(name='KDE',
                                x=xr, 
                                y=kdy, 
                                mode='lines', 
                                marker_color='#D2691E'))

        fig.add_trace(go.Scatter(name="Peaks", x=close_avg_peak, y=volume_pd_peak, 
                                mode='markers', 
                                marker=dict(size=10, color="MediumPurple")))

        fig.add_trace(go.Scatter(name="Poc", 
                                x=peak_df[peak_df['poc_order'] == 1].close_avg, 
                                y=peak_df[peak_df['poc_order'] == 1].volume_prob_d, 
                                mode='markers',
                                text=[peak_df[peak_df['poc_order'] == 1].poc_order],
                                marker=dict(color='LightSkyBlue',
                                                size=10,
                                                line=dict(color='blue',
                                                        width=1)))
        )

        fig.update_layout(
                title=dict(text='Fixed Range Volume Profile (Daily over 2 Months)', x=0.5),
                xaxis_title="Average of Close",
                yaxis_title="Probability Density of Volume",
                xaxis=dict(zeroline=False, showgrid=False), 
                yaxis=dict(zeroline=False, showgrid=False)
                )

        fig.add_vrect(name="Current Close Price",
                        x0=current_close_value,
                        x1=current_close_value,
                        fillcolor='orange',
                        opacity=1,
                        line_width=1
                        )
        
        # fig.add_vrect(name="Support Line",
        #                 x0=support,
        #                 x1=support,
        #                 fillcolor='orange',
        #                 opacity=1,
        #                 line_width=3
        #                 )
        
        # fig.add_vrect(name="Resistence Line",
        #                 x0=resistence,
        #                 x1=resistence,
        #                 fillcolor='orange',
        #                 opacity=1,
        #                 line_width=3
        #                 )
        
        fig.show()

        return peak_df

get_volume_profile(data)

,close_avg,volume_prob_d,peakvall_order,poc_order
0,1.073648,3.968310,1.0,5.0
1,1.086799,5.737237,2.0,2.0
2,1.101829,5.117344,3.0,3.0
3,1.120617,7.381820,4.0,1.0
4,1.178858,5.052294,5.0,4.0
5,1.229585,2.906322,6.0,6.0
6,1.293463,2.375532,7.0,7.0
7,1.359219,2.114535,8.0,8.0


### <font color='white'> __3.8 Fibonacci Levels__</font> <a class="anchor" id="threeseight"></a>

[<font color='mediumpurple'>Back to Index </font>](#index)


__Notes:__
* https://eodhistoricaldata.com/financial-academy/technical-analysis-examples/fibonacci-sequence-in-trading-with-python/

* https://www.jumpstarttrading.com/fibonacci-retracement/
>* After you have identified the most recent swing high and swing low you’re going to select your Fib Retracement Tool. 
>* Allways apply on highs and Lows, not on closes and opens
>* Applying Fibonacci retracements over a short timeframe is ineffective. The shorter the timeframe, the less reliable the retracement levels.

In [ ]:
def fibonacci_retracement_levels(data):
        
        current_price = data[['open', 'high', 'low', 'close']].iloc[-1].close
        
        # Measure the simple levels
        min_level = data['low'].min()
        max_level = data['high'].max()

        # Get the levels Index
        min_index = data['low'].idxmin()
        max_index = data['high'].idxmax()

        # If necessary to check the rows of each
        # min_row =data.iloc[min_index]
        # max_row =data.iloc[max_index]

        # Movement is Descendent 
        if max_index < min_index:
            
            Diff = max_level - min_level
            Fib100 = max_level
            Fib764 = min_level + (Diff * 0.764)
            Fib618 = min_level + (Diff * 0.618)
            Fib50 = min_level + (Diff * 0.5)
            Fib382 = min_level + (Diff * 0.382)
            Fib236 = min_level + (Diff * 0.236)
            Fib0 = min_level

            # Get Relevance 

            fib_levels = [Fib100, Fib764, Fib618, Fib50, Fib382, Fib236, Fib0]

                # Calculate absolute dif between current_price and each value
            abs_dif = [abs(current_price - i) for i in fib_levels]

                # Find the index of the minimum difference
            min_index = abs_dif.index(min(abs_dif))
                
                # Get the value with the minimum difference
            fibo_relevance = fib_levels[min_index]

                #Fibo order
            fibo_order = 'Descendent'
            fibo_signal = 'Buy'
            
            fibo_relevance = fibo_relevance
            Fib0 = Fib0
            Fib100 = Fib100 
            fibo_order = fibo_order
            fibo_signal = fibo_signal 
        
        # Movement is Ascendent
        elif min_index < max_index:
            
            Diff = max_level - min_level
            Fib100 = min_level
            Fib764 = max_index - (Diff * 0.764)
            Fib618 = max_index - (Diff * 0.618)
            Fib50 = max_index - (Diff * 0.5)
            Fib382 = max_index - (Diff * 0.382)
            Fib236 = max_index - (Diff * 0.236)
            Fib0 = max_index

            # Get Relevance 
                # Values

            fib_levels = [Fib100, Fib764, Fib618, Fib50, Fib382, Fib236, Fib0]

                # Calculate absolute dif between current_price and each value
            abs_dif = [abs(current_price - i) for i in fib_levels]

                # Find the index of the minimum difference
            min_index = abs_dif.index(min(abs_dif))

                # Get the value with the minimum difference
            fibo_relevance = fib_levels[min_index]
                #Fibo order
            fibo_order = 'Ascendent'
            fibo_signal = 'Sell'
            

            fibo_relevance = fibo_relevance
            Fib0 = Fib0
            Fib100 = Fib100 
            fibo_order = fibo_order
            fibo_signal = fibo_signal
            
        result_df = pd.DataFrame(columns=['Function', 'Signal', 'Relevance'])
            
        result_df = pd.concat([result_df, pd.DataFrame({
                                        'Function': ['Fibonacci'],
                                        'Signal': [fibo_signal],
                                        'Relevance': [fibo_relevance]
                                    })], ignore_index=True) 

        return result_df, fibo_order, Fib100, Fib764, Fib618, Fib50, Fib382, Fib236, Fib0



In [ ]:
result_df, fibo_order, Fib100, Fib764, Fib618, Fib50, Fib382, Fib236, Fib0 = fibonacci_retracement_levels(data)

In [ ]:
data['Fib100'] = Fib100
data['Fib764'] = Fib764
data['Fib618'] = Fib618
data['Fib50'] = Fib50
data['Fib382'] = Fib382
data['Fib236'] = Fib236
data['Fib0'] = Fib0

In [ ]:
# fig1 = go.Figure(data=[go.Candlestick(x=data.index,
#                                         open=data['open'],
#                                         high=data['high'],
#                                         low=data['low'],
#                                         close=data['close'])
#                         ])

# # Fibonacci levels
# fib_levels = [Fib100, Fib764, Fib618, Fib50, Fib382, Fib236, Fib0]

# # Add Fibonacci levels as horizontal lines
# for level in fib_levels:
#     fig1.add_shape(type="line",
#                     xref="x", yref="y",
#                     x0=data.index[0], y0=level,
#                     x1=data.index[-1], y1=level,
#                     line=dict(color="red", width=1)
#                     )

# fig1.update_layout(title=dict(text='Daily Base Fibonacci Levels {}'.format(fibo_order),
#                                 x=.5  # Title relative position according to the x-axis, range (0,1)
#                                 ))

# fig1.show()


## <font color='white'> __4. BACKTESTING Strategies__ </font> <a class="anchor" id="four"></a>

[<font color='mediumpurple'>Back to Index </font>](#index)

In [74]:
data_backtest = data[['time','open','high','low','close','volume']].copy()

data_backtest.rename(columns={'time':'Time','open':'Open','high':'High','low':'Low','close':'Close','volume':'Volume'}, inplace=True)
data_backtest

,Time,Open,High,Low,Close,Volume
0,1985-07-26 00:00:00+00:00,0.68170,0.68300,0.67970,0.68190,641
1,1985-07-29 00:00:00+00:00,0.69180,0.69270,0.68980,0.69150,551
2,1985-07-30 00:00:00+00:00,0.69310,0.69460,0.69020,0.69300,871
3,1985-07-31 00:00:00+00:00,0.69950,0.70080,0.69850,0.69940,451
4,1985-08-01 00:00:00+00:00,0.69590,0.70150,0.69280,0.69570,1721
...,...,...,...,...,...,...
9995,2024-02-13 00:00:00+00:00,1.07711,1.07949,1.07008,1.07091,56846
9996,2024-02-14 00:00:00+00:00,1.07075,1.07346,1.06951,1.07276,53500
9997,2024-02-15 00:00:00+00:00,1.07249,1.07849,1.07240,1.07722,51996
9998,2024-02-16 00:00:00+00:00,1.07650,1.07876,1.07321,1.07763,50485


In [75]:
class MyCrossoverStrat(Strategy):
    FirstEma = 13
    SecondEma = 48
    ThirdEma = 200

    def init(self):
        self.ema13 = self.I(pta.ema, data_backtest.Close, self.FirstEma)
        self.ema48 = self.I(pta.ema, data_backtest.Close, self.SecondEma)
        self.ema200 = self.I(pta.ema, data_backtest.Close, self.ThirdEma)

    def next(self):
        if (self.ema13 < self.ema48 < self.ema200):
            if self.position.is_long or not self.position:
                self.position.close()
                self.sell(size=1)

        elif self.ema13 > self.ema48 > self.ema200:
            if self.position.is_short or not self.position:
                self.position.close()
                self.buy(size=1)

cross_bt = Backtest(data=data_backtest, strategy=MyCrossoverStrat, cash=200_000, commission=0.00, exclusive_orders=True)
print('OPTIMIZING EMA Crossovers METRICS...')

# Optimization Params
optimizer = 'Win Rate [%]'

cross_opt = cross_bt.optimize(
    FirstEma = range(5, 25, 1),
    SecondEma = range(26, 80, 1),
    ThirdEma = range(81, 200, 1),
    maximize = optimizer,
    max_tries = 200
)
print(f'BEST_FIRST_PERIOD_Ema_Cross = {cross_opt._strategy.FirstEma} \n BEST_SECOND_PERIOD_Ema_Cross = {cross_opt._strategy.SecondEma} \n BEST_THIRD_PERIOD_Ema_Cross {cross_opt._strategy.ThirdEma}')

cross_bt.plot()

cross_bt.run()
    

C:\Users\joaom\AppData\Local\Temp\ipykernel_21536\464388951.py:22: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



OPTIMIZING EMA Crossovers METRICS...


  0%|          | 0/17 [00:00<?, ?it/s]

BEST_FIRST_PERIOD_Ema_Cross = 10 
 BEST_SECOND_PERIOD_Ema_Cross = 65 
 BEST_THIRD_PERIOD_Ema_Cross 109


GridPlot(id='p1295', ...)

Start                                     0.0
End                                    9999.0
Duration                               9999.0
Exposure Time [%]                       97.99
Equity Final [$]                 200001.05322
Equity Peak [$]                  200001.23886
Return [%]                           0.000527
Buy & Hold Return [%]               58.719754
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.000209
Avg. Drawdown [%]                   -0.000015
Max. Drawdown Duration                 3145.0
Avg. Drawdown Duration              79.336066
# Trades                                 47.0
Win Rate [%]                        36.170213
Best Trade [%]                       36.17416
Worst Trade [%]                      -6.33416
Avg. Trade [%]                    

In [79]:
class MyBandsStrat(Strategy):
    Length = 14
    Std = 2

    def init(self):
        # Calculate Bollinger Bands
        self.bbands_data = self.I(pta.bbands, data_backtest.Close, self.Length, self.Std)
        
        # Check if the array is not empty or contains only NaN values
        if np.any(~np.isnan(self.bbands_data[:, 3])):
            self.bbb_col_mean = np.nanmean(self.bbands_data[:, 3])
        else:
            self.bbb_col_mean = np.nan  # Set mean to NaN if the array is empty or contains only NaN values

    def next(self):
        # Check if the array is not empty or contains only NaN values
        if np.any(~np.isnan(self.bbands_data[:, 3])):
            if self.bbands_data[:, 3] > self.bbb_col_mean:
                if self.data_backtest.Close < self.bbands_data[:, 0]:
                    self.position.close()
                    self.buy(size=1)  # Buy
                elif self.data_backtest.Close > self.bbands_data[:, 2]:
                    self.position.close()
                    self.sell(size=1)  # Sell

bb_bt = Backtest(data=data_backtest, strategy=MyBandsStrat, cash=200_000, commission=0.00, exclusive_orders=True)
print('OPTIMIZING Bollinger Bands METRIC...')

# Optimization Params
optimizer = 'Win Rate [%]'

bb_opt = bb_bt.optimize(
    Length = range(8, 25, 1),
    Std = range(1, 10, 1),
    maximize= optimizer, #optimizer_params,
    max_tries=200
)

print(f'BEST_FIRST_Length = {bb_opt._strategy.Length} \n BEST_SECOND_Std = {bb_opt._strategy.Std}')

bb_bt.plot()

bb_bt.run()

C:\Users\joaom\AppData\Local\Temp\ipykernel_21536\1708354970.py:26: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



OPTIMIZING Bollinger Bands METRIC...


  0%|          | 0/17 [00:00<?, ?it/s]

c:\Users\joaom\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\backtesting.py:1384: FutureWarning:

The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError



BEST_FIRST_Length = 8 
 BEST_SECOND_Std = 1


Start                                     0.0
End                                    9999.0
Duration                               9999.0
Exposure Time [%]                         0.0
Equity Final [$]                     200000.0
Equity Peak [$]                      200000.0
Return [%]                                0.0
Buy & Hold Return [%]               58.719754
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                  0.0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [80]:
class MyRsiStrat(Strategy):
    Overbought_level = 70
    Oversold_level =  30
    Length = 2

    def init(self):
        # Calculate Bollinger Bands
        self.rsi = self.I(pta.rsi, data_backtest.Close, self.Length)
        
    def next(self):
        if self.rsi < self.Oversold_level:
            if self.position.is_long or not self.position:
                self.position.close()
                self.buy(size=1)  # Buy
        elif self.rsi > self.Overbought_level:
            if self.position.is_short or not self.position:
                self.position.close()
                self.sell(size=1)  # Sell

def custom_optimizer_params(strategy):
    if strategy._strategy.Overbought_level + strategy._strategy.Oversold_level != 100:
        return -1

    return strategy['Win Rate [%]']

rsi_bt = Backtest(data=data_backtest, strategy=MyRsiStrat, cash=200_000, commission=0.00, exclusive_orders=True)
print('OPTIMIZING Rsi METRIC...')

# Optimization Params
optimizer = 'Win Rate [%]'

rsi_opt = rsi_bt.optimize(
    Length = range(8, 20, 1),
    Overbought_level = range(60, 80, 5),
    Oversold_level = range(20, 40, 5),
    maximize=custom_optimizer_params,
    max_tries=200
)

print(f'BEST_FIRST_Length = {rsi_opt._strategy.Length} \n BEST_SECOND_Overbought_level = {rsi_opt._strategy.Overbought_level} \n BEST_THIRD_Oversold_level = {rsi_opt._strategy.Oversold_level}')

rsi_bt.plot()

rsi_bt.run()

OPTIMIZING Rsi METRIC...


C:\Users\joaom\AppData\Local\Temp\ipykernel_21536\673954051.py:26: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



  0%|          | 0/16 [00:00<?, ?it/s]

BEST_FIRST_Length = 14 
 BEST_SECOND_Overbought_level = 65 
 BEST_THIRD_Oversold_level = 35


Start                                     0.0
End                                    9999.0
Duration                               9999.0
Exposure Time [%]                       99.96
Equity Final [$]                 199999.61359
Equity Peak [$]                   200000.0337
Return [%]                          -0.000193
Buy & Hold Return [%]               58.719754
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.000469
Avg. Drawdown [%]                   -0.000097
Max. Drawdown Duration                 9966.0
Avg. Drawdown Duration                 1998.2
# Trades                               3299.0
Win Rate [%]                        34.768112
Best Trade [%]                      13.319536
Worst Trade [%]                     -5.452384
Avg. Trade [%]                    

# BACKTEST

## __Duvidas__

Explicar tudo!!!

__Indicador Fixed Range Volume Index__

# Devo apresentar me tese também a criação e implementação da metrica Fixed Range Volume Index??? Ou basta simplesmente fundamentar a teoria da metrica?

## Devo ter os ultimos 2 meses desde o inicio do mes ou bastam ultimo 60 dias??? Estudar !!!
## Used Kernel Density Estimator varia conforme Ação e coforme TimeLine | Falta testar para outras stocks para alem da GOOGLE


Rotinas proprias e ja desenvolvidas

__Duvidas Gerais__

# Começo por testar explicar o que é uma time series e suas componentes (Observed, Trend, Seasonal, Residuals) 

# Teste de estaridade da time series, todos os testes detalhados? 

## primeiro stationar
## Raizes unitarias testes para determinar 

## Secção 3 metodoligia, procedimentos, impiricos 
## Secção 4 apresentado, 
## Secção 5 Descriçºao critica dos acontecimentos (em que varia, implicação)

# Arquivo Jorge Bravo

Referencia de quais o metodos dados, avanço dos resultados , abstrato alargado, captar o leitor desde o inicio.
# CMVM consultores de investimento | Regulamento diretiva de consultores de investimento | Regulamento cmvm 3 2021 pode ter atualizar  


## Tem um função que detecta se o p-value < 0,005 - Stationary | > 0,005 - Non Stationary
## (ADF) Augmented Dickey-Fuller -> ao detalhe?
## Sendo que é aplicado a stock e cambiais, cujos são por defeito não Stationarios devo apresentar aplicação a casos stationarios ?? (( Implementado na Matris))
## Não engloba (PACF) Partial autocorrelation function, pode ser implementado se necessario, contudo aquando aplicada a primeira "Differenciação" passa a ser classificado com Random Walk o que é verdade.

### Não sendo Staci. vale a pena apresentar o quando é inutil a utilização de modelos como Arima, Sarima???



